# Backward induction

In [10]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. load packages 
import numpy as np

# c. local modules
from backward_induction import backward_induction_class

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulate data

In [54]:
# define model from class
model = backward_induction_class()

# expected value 0 
ev0 = np.zeros(model.n)






In [55]:
# simulate data (pk skal enten være 1 eller 0)
simdata = model.sim_data()


AttributeError: 'backward_induction_class' object has no attribute 'pk'